In [10]:
import sys

from peft import LoraConfig

sys.path.append("../")
from src import AutoStableDiffusionModel, LoraWrapper, convert_to_lora_target_names

In [2]:
def print_dash_line(length=100):
    print("-" * length)


In [3]:
def print_number_of_trainable_model_params(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    
    print(f"Trainable model parameters: {trainable_model_params}\nAll model parameters: {all_model_params}\nPercentage of trainable parameters: {trainable_model_params/all_model_params*100:.2f}%")

In [4]:
def print_lora_params_and_config(base_model_name, unet_config, text_encoder_config):
    base_model = AutoStableDiffusionModel.from_pretrained(base_model_name)
    
    if unet_config["r"] != 0:
        unet_lora_config = LoraConfig(
            r=unet_config["r"],
            lora_alpha=unet_config["alpha"],
            target_modules=convert_to_lora_target_names(unet_config["targets"], "unet"),
            init_lora_weights="gaussian",
        )
    else:
        unet_lora_config = None
    
    if text_encoder_config["r"] != 0:
        te_lora_config = LoraConfig(
            r=text_encoder_config["r"],
            lora_alpha=text_encoder_config["alpha"],
            target_modules=convert_to_lora_target_names(text_encoder_config["targets"], "text_encoder"),
            init_lora_weights="gaussian",
        )
    else:
        te_lora_config = None
    
    sdxl_lora_model = LoraWrapper.from_config(base_model, unet_lora_config, te_lora_config)
    
    print("Configuration:")
    if unet_lora_config:
        print(f"UNet - r: {unet_config['r']}, alpha: {unet_config['alpha']}, targets: {unet_config['targets']}")
    else:
        print("UNet - LoRA not applied")
        
    if te_lora_config:
        print(f"Text Encoder - r: {text_encoder_config['r']}, alpha: {text_encoder_config['alpha']}, targets: {text_encoder_config['targets']}")
    else:
        print("Text Encoder - LoRA not applied")
    
    print("\nParameters Detail:")
    print_number_of_trainable_model_params(sdxl_lora_model)

# Experiment 1 Configurations

In [5]:
sdxl_base_model_name = "stabilityai/stable-diffusion-xl-base-1.0"

In [6]:
sdxl_experiment1_configs = [
    {
        "unet": {"r": 2, "alpha": 1, "targets": ["k", "q", "v", "o"]},
        "text_encoder": {"r": 0, "alpha": 0, "targets": []}
    },
    {
        "unet": {"r": 1, "alpha": 1, "targets": ["k", "q", "v", "o"]},
        "text_encoder": {"r": 4, "alpha": 1, "targets": ["k", "q", "v", "o"]}
    },
    {
        "unet": {"r": 0, "alpha": 0, "targets": []},
        "text_encoder": {"r": 8, "alpha": 1, "targets": ["k", "q", "v", "o"]}
    }
]

In [7]:
for i, config in enumerate(sdxl_experiment1_configs):
    print_dash_line()
    print(f"Configuration {i + 1}")
    
    unet_config = config["unet"]
    text_encoder_config = config["text_encoder"]
    
    print_lora_params_and_config(sdxl_base_model_name, unet_config, text_encoder_config)

----------------------------------------------------------------------------------------------------
Configuration 1
Configuration:
UNet - r: 2, alpha: 1, targets: ['k', 'q', 'v', 'o']
Text Encoder - LoRA not applied

Parameters Detail:
Trainable model parameters: 2903040
All model parameters: 3471740907
Percentage of trainable parameters: 0.08%
----------------------------------------------------------------------------------------------------
Configuration 2
Configuration:
UNet - r: 1, alpha: 1, targets: ['k', 'q', 'v', 'o']
Text Encoder - r: 4, alpha: 1, targets: ['k', 'q', 'v', 'o']

Parameters Detail:
Trainable model parameters: 3057152
All model parameters: 3471895019
Percentage of trainable parameters: 0.09%
----------------------------------------------------------------------------------------------------
Configuration 3
Configuration:
UNet - LoRA not applied
Text Encoder - r: 8, alpha: 1, targets: ['k', 'q', 'v', 'o']

Parameters Detail:
Trainable model parameters: 3211264
Al

# Experiment 2 Configurations

In [8]:
sdxl_experiment2_configs = [
    {
        "unet": {"r": 1, "alpha": 1, "targets": ["k", "q", "v", "o"]},
        "text_encoder": {"r": 0, "alpha": 0, "targets": []}
    },
    {
        "unet": {"r": 2, "alpha": 1, "targets": ["k", "q", "v", "o"]},
        "text_encoder": {"r": 0, "alpha": 0, "targets": []}
    },
    {
        "unet": {"r": 4, "alpha": 1, "targets": ["k", "q", "v", "o"]},
        "text_encoder": {"r": 0, "alpha": 0, "targets": []}
    },
    {
        "unet": {"r": 8, "alpha": 1, "targets": ["k", "q", "v", "o"]},
        "text_encoder": {"r": 0, "alpha": 0, "targets": []}
    },
    {
        "unet": {"r": 64, "alpha": 1, "targets": ["k", "q", "v", "o"]},
        "text_encoder": {"r": 0, "alpha": 0, "targets": []}
    },
]

In [9]:
for i, config in enumerate(sdxl_experiment2_configs):
    print_dash_line()
    print(f"Configuration {i + 1}")
    
    unet_config = config["unet"]
    text_encoder_config = config["text_encoder"]
    
    print_lora_params_and_config(sdxl_base_model_name, unet_config, text_encoder_config)

----------------------------------------------------------------------------------------------------
Configuration 1
Configuration:
UNet - r: 1, alpha: 1, targets: ['k', 'q', 'v', 'o']
Text Encoder - LoRA not applied

Parameters Detail:
Trainable model parameters: 1451520
All model parameters: 3470289387
Percentage of trainable parameters: 0.04%
----------------------------------------------------------------------------------------------------
Configuration 2
Configuration:
UNet - r: 2, alpha: 1, targets: ['k', 'q', 'v', 'o']
Text Encoder - LoRA not applied

Parameters Detail:
Trainable model parameters: 2903040
All model parameters: 3471740907
Percentage of trainable parameters: 0.08%
----------------------------------------------------------------------------------------------------
Configuration 3
Configuration:
UNet - r: 4, alpha: 1, targets: ['k', 'q', 'v', 'o']
Text Encoder - LoRA not applied

Parameters Detail:
Trainable model parameters: 5806080
All model parameters: 347464394